In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Just a formatting related plugin
%load_ext nb_black

<IPython.core.display.Javascript object>

# Using the final class to run the starspace training

In [3]:
import sys

sys.path.append("../")

<IPython.core.display.Javascript object>

In [4]:
from snpe.embeddings.starspace_class import StarSpaceEmbedder

<IPython.core.display.Javascript object>

In [5]:
embedder = StarSpaceEmbedder()

<IPython.core.display.Javascript object>

In [6]:
embedder.process_input_data()

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Loaded input data of shape (3204851, 13)
Train set size: (88312,)
Test set size: (22079,)
Saving train and test datasets


<IPython.core.display.Javascript object>

In [ ]:
embedder.train_starspace()

StarSpace command to be run: 
 /StarSpace/starspace train -trainFile /data/reputation-systems/snpe/artifacts/marketplace/pageview_train.txt -validationFile /data/reputation-systems/snpe/artifacts/marketplace/pageview_test.txt -label product -model /data/reputation-systems/snpe/artifacts/marketplace/productspace -trainMode 1 -epoch 100 -validationPatience 10 -thread 16
Arguments: 
lr: 0.01
dim: 100
epoch: 100
maxTrainTime: 8640000
validationPatience: 10
saveEveryEpoch: 0
loss: hinge
margin: 0.05
similarity: cosine
maxNegSamples: 10
negSearchLimit: 50
batchSize: 5
thread: 16
minCount: 1
minCountLabel: 1
label: product
label: product
ngrams: 1
bucket: 2000000
adagrad: 1
trainMode: 1
fileFormat: fastText
normalizeText: 0
dropoutLHS: 0
dropoutRHS: 0
useWeight: 0
weightSep: :
Start to initialize starspace model.
Build dict from input file : /data/reputation-systems/snpe/artifacts/marketplace/pageview_train.txt
Loading data from file : /data/reputation-systems/snpe/artifacts/marketplace/pagev

# Testing starspace command line tool (rough)

In [16]:
import pyreadr
import numpy as np

<IPython.core.display.Javascript object>

In [4]:
pageviews = pyreadr.read_r("../artifacts/marketplace/pageviews_data_bazaarvoice.Rds")
pageviews = pageviews[None]
pageviews.head()

,productid,ProductsSeen,hasheduaip,ProductAvgReview,ProductNumberOfReviews,category_name_viewed,categoryid_of_viewed,category_name_focal,categoryid_of_focal
0,1017360,1017360,,4.3411,214,TV remote controls,33015078,TV remote controls,33015078
1,1017360,1076198,,4.3411,214,TV remote controls,33015078,TV remote controls,33015078
2,1017470,326858,,3.9091,44,Docking stations and speakers,36065095,Docking stations and speakers,36065095
3,1017470,1017470,,3.9091,44,Docking stations and speakers,36065095,Docking stations and speakers,36065095
4,1017470,1085567,,3.9091,44,Docking stations and speakers,36065095,Docking stations and speakers,36065095


<IPython.core.display.Javascript object>

In [5]:
import pandas as pd

<IPython.core.display.Javascript object>

**We decided to produce starspace embeddings for all products in our dataset (not a single category), but fit a distribution (eg. Normalizing Flow) over only the products in the category we are interested in. We can use the embeddings of all the other products to train a neural network to predict product quality (aka final distribution of ratings) from product_embeddings**

In [6]:
pageview_summaries = pd.read_csv(
    "../artifacts/marketplace/pageview_summaries.txt", sep="\t"
)
pageview_summaries.head()

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,productid,VisitTimeStamp,ThisProductPurchased,ProductPrice,ProductAvgReview,ProductNumberOfReviews,bvid,hasheduaip,loadid,type,IDofMostRecentReview,WhetherMostRecentReviewWasSeen,ProductsSeen
0,3550887.0,2015-03-14 13:05:09,f,\N,0.0000,0.0,00001e38-2c31-479e-b9ee-0078bd38244e,NaN,e49d814b3f03f1cd,Product,47118474,t,"3550887,2399166,2476773"
1,3550887.0,2015-03-14 13:05:09,f,\N,NaN,NaN,00001e38-2c31-479e-b9ee-0078bd38244e,NaN,e49d814b3f03f1cd,Product,47118474,t,"3550887,2399166,2476773"
2,2399166.0,2015-03-14 13:04:13,f,\N,4.0000,3.0,00001e38-2c31-479e-b9ee-0078bd38244e,NaN,406e5ed51bb94425,Product,42593044,t,"3550887,2399166,2476773"
3,2399166.0,2015-03-14 13:04:13,f,\N,NaN,NaN,00001e38-2c31-479e-b9ee-0078bd38244e,NaN,406e5ed51bb94425,Product,42593044,t,"3550887,2399166,2476773"
4,2476773.0,2015-03-14 12:52:48,f,249.99,4.3333,3.0,00001e38-2c31-479e-b9ee-0078bd38244e,NaN,badbb477b8d204d1,Product,47115055,f,"3550887,2399166,2476773"


<IPython.core.display.Javascript object>

In [7]:
pageview_summaries.shape

(3204851, 13)

<IPython.core.display.Javascript object>

In [8]:
pageview_summaries.bvid.isnull().mean()

0.3683503538854068

<IPython.core.display.Javascript object>

In [9]:
pageview_summaries.ProductsSeen.isnull().mean()

0.0

<IPython.core.display.Javascript object>

In [10]:
pageview_summaries["ProductsSeen"].unique().shape[0] / pageview_summaries.shape[0]

0.034444971076658476

<IPython.core.display.Javascript object>

In [11]:
pageview_summaries.head(10)

,productid,VisitTimeStamp,ThisProductPurchased,ProductPrice,ProductAvgReview,ProductNumberOfReviews,bvid,hasheduaip,loadid,type,IDofMostRecentReview,WhetherMostRecentReviewWasSeen,ProductsSeen
0,3550887.0,2015-03-14 13:05:09,f,\N,0.0000,0.0,00001e38-2c31-479e-b9ee-0078bd38244e,NaN,e49d814b3f03f1cd,Product,47118474,t,"3550887,2399166,2476773"
1,3550887.0,2015-03-14 13:05:09,f,\N,NaN,NaN,00001e38-2c31-479e-b9ee-0078bd38244e,NaN,e49d814b3f03f1cd,Product,47118474,t,"3550887,2399166,2476773"
2,2399166.0,2015-03-14 13:04:13,f,\N,4.0000,3.0,00001e38-2c31-479e-b9ee-0078bd38244e,NaN,406e5ed51bb94425,Product,42593044,t,"3550887,2399166,2476773"
3,2399166.0,2015-03-14 13:04:13,f,\N,NaN,NaN,00001e38-2c31-479e-b9ee-0078bd38244e,NaN,406e5ed51bb94425,Product,42593044,t,"3550887,2399166,2476773"
4,2476773.0,2015-03-14 12:52:48,f,249.99,4.3333,3.0,00001e38-2c31-479e-b9ee-0078bd38244e,NaN,badbb477b8d204d1,Product,47115055,f,"3550887,2399166,2476773"
5,2476773.0,2015-03-14 12:52:48,f,249.99,NaN,NaN,00001e38-2c31-479e-b9ee-0078bd38244e,NaN,badbb477b8d204d1,Product,47115055,f,"3550887,2399166,2476773"
6,3191040.0,2015-03-04 13:25:22,f,269.99,4.3333,33.0,00004e69-01d5-4f8a-89e2-e2d5a40836f9,NaN,731e25598da004834,Product,46613093,f,"1374467,1393840,3191040"
7,3191040.0,2015-03-04 13:25:22,f,269.99,NaN,NaN,00004e69-01d5-4f8a-89e2-e2d5a40836f9,NaN,731e25598da004834,Product,46613093,f,"1374467,1393840,3191040"
8,NaN,2015-03-04 13:42:14,f,\N,NaN,NaN,00004e69-01d5-4f8a-89e2-e2d5a40836f9,NaN,b9ce352f09ca46629,Category,\N,f,"1374467,1393840,3191040"
9,NaN,2015-03-04 13:27:21,f,\N,NaN,NaN,00004e69-01d5-4f8a-89e2-e2d5a40836f9,NaN,77f90417db27b970178d,Category,\N,f,"1374467,1393840,3191040"


<IPython.core.display.Javascript object>

In [12]:
pd.Series(pageview_summaries["ProductsSeen"].unique()).str.split(",").apply(
    lambda x: ["product_" + str(token) for token in x]
).str.join(" ")

0           product_3550887 product_2399166 product_2476773
1           product_1374467 product_1393840 product_3191040
2                                           product_2441715
3                           product_6523811 product_2952266
4           product_8900645 product_1702462 product_3796586
                                ...                        
110386      product_4871699 product_4871778 product_4764744
110387                      product_3224412 product_3225947
110388      product_9015838 product_6249102 product_9015814
110389    product_3596102 product_3315819 product_339349...
110390    product_2764636 product_3214532 product_152942...
Length: 110391, dtype: object

<IPython.core.display.Javascript object>

In [13]:
pageview_strings = (
    pd.Series(pageview_summaries["ProductsSeen"].unique())
    .str.split(",")
    .apply(lambda x: ["product_" + str(token) for token in x])
    .str.join(" ")
)

<IPython.core.display.Javascript object>

In [13]:
from sklearn.model_selection import train_test_split

<IPython.core.display.Javascript object>

In [14]:
pageview_train, pageview_test = train_test_split(pageview_strings, test_size=0.2)
pageview_train.shape, pageview_test.shape

((88312,), (22079,))

<IPython.core.display.Javascript object>

In [15]:
pageview_strings.to_csv(
    "../artifacts/marketplace/pageview_strings.txt", index=False, encoding="utf-8"
)
pageview_train.to_csv(
    "../artifacts/marketplace/pageview_train.txt", index=False, encoding="utf-8"
)
pageview_test.to_csv(
    "../artifacts/marketplace/pageview_test.txt", index=False, encoding="utf-8"
)

<IPython.core.display.Javascript object>

In [26]:
a = np.array(
    pd.Series(pageview_summaries["ProductsSeen"].unique())
    .str.split(",")
    .apply(lambda x: np.array(x).astype("int"))
)

<IPython.core.display.Javascript object>

In [27]:
a = np.concatenate(a)
a.shape

(478749,)

<IPython.core.display.Javascript object>

In [29]:
np.unique(a).shape

(32933,)

<IPython.core.display.Javascript object>

# Checking artifact deposited by StarSpace

**Starspace command**

./starspace train -trainFile /media/glia/reputation-systems/snpe/artifacts/marketplace/pageview_train.txt -model productspace -label 'product' -trainMode 1 -epoch 100 -validationPatience 10 -validationFile /media/glia/reputation-systems/snpe/artifacts/marketplace/pageview_test.txt -thread 16


Lots and lots of parameters, probably need to set up a model tuning code

In [47]:
product_embeddings = pd.read_csv(
    "../artifacts/marketplace/productspace.tsv", sep="\t", header=None
)

<IPython.core.display.Javascript object>

In [48]:
product_embeddings.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,product_2398095,-0.000342,0.001594,0.001381,-0.001720,0.000024,0.001139,-0.003441,0.002121,0.000136,...,0.000377,-0.003633,-0.002781,-0.000685,-0.000697,0.000008,0.003361,-0.000343,-0.001666,-0.001669
1,product_2629881,-0.003545,-0.001928,0.002686,-0.004293,0.002870,0.000466,-0.002480,0.001578,-0.000770,...,-0.000536,-0.000995,0.001846,-0.001848,0.000751,-0.001648,-0.001224,-0.002357,-0.002670,0.005191
2,product_2457747,0.002997,0.001438,0.001200,0.002703,0.000953,-0.001649,-0.001580,0.000305,0.001011,...,0.000090,-0.001305,0.000379,-0.001880,0.001493,0.002100,-0.000627,0.000235,0.000883,0.002010
3,product_2621353,0.000150,-0.002810,0.000399,0.000554,0.002330,-0.001891,-0.000404,-0.000718,0.002021,...,0.002116,-0.000727,-0.000577,-0.000851,0.000717,0.001625,0.000882,0.002180,0.001483,-0.002198
4,product_2598651,-0.001935,-0.001259,0.002641,-0.000337,0.000513,-0.006381,0.001413,-0.001804,0.000256,...,0.000438,0.002462,0.003383,-0.000635,0.002466,-0.004213,0.004412,0.000639,0.003211,-0.000423


<IPython.core.display.Javascript object>

In [49]:
product_embeddings.shape

(31726, 101)

<IPython.core.display.Javascript object>